### m8.1stan

m8.1stan is the first model in the Statistical Rethinking book (pp. 249) using Stan.

Here we will use Turing's NUTS support, which is currently (2018) the originalNUTS by [Hoffman & Gelman]( http://www.stat.columbia.edu/~gelman/research/published/nuts.pdf) and not the one that's in Stan 2.18.2, i.e., Appendix A.5 in: https://arxiv.org/abs/1701.02434

The TuringModels pkg imports modules such as CSV and DataFrames

In [1]:
using TuringModels, Turing

Turing.setadbackend(:reverse_diff);
Turing.turnprogress(false);

┌ Info: [Turing]: global PROGRESS is set as false
└ @ Turing /Users/rob/.julia/packages/Turing/UXxKz/src/Turing.jl:24


Read in the `rugged` data as a DataFrame

In [2]:
d = CSV.read(rel_path("..", "data", "rugged.csv"), delim=';');

Show size of the DataFrame (should be 234x51)

In [3]:
size(d)

(234, 51)

Apply log() to each element in rgdppc_2000 column and add it as a new column

In [4]:
d = hcat(d, map(log, d[Symbol("rgdppc_2000")]));

Rename our col x1 => log_gdp

In [5]:
rename!(d, :x1 => :log_gdp);

Now we need to drop every row where rgdppc_2000 == missing

When this (https://github.com/JuliaData/DataFrames.jl/pull/1546) hits DataFrame it'll be conceptually easier: i.e., completecases!(d, :rgdppc_2000)

In [6]:
notisnan(e) = !ismissing(e)
dd = d[map(notisnan, d[:rgdppc_2000]), :];

Updated DataFrame dd size (should equal 170 x 52)

In [7]:
size(dd)

(170, 52)

Define the Turing model

In [8]:
@model m8_1stan(y, x₁, x₂) = begin
    σ ~ Truncated(Cauchy(0, 2), 0, Inf)
    βR ~ Normal(0, 10)
    βA ~ Normal(0, 10)
    βAR ~ Normal(0, 10)
    α ~ Normal(0, 100)

    for i ∈ 1:length(y)
        y[i] ~ Normal(α + βR * x₁[i] + βA * x₂[i] + βAR * x₁[i] * x₂[i], σ)
    end
end;

Test to see that the model is sane. Use 2000 for now, as in the book.
Need to set the same stepsize and adapt_delta as in Stan...

Use Turing mcmc

In [9]:
posterior = sample(m8_1stan(dd[:log_gdp], dd[:rugged], dd[:cont_africa]),
Turing.NUTS(2000, 1000, 0.95));

┌ Info: [Turing] looking for good initial eps...
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/support/hmc_core.jl:240
[NUTS{Turing.Core.FluxTrackerAD,Union{}}] found initial ϵ: 0.1
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/support/hmc_core.jl:235
┌ Info:  Adapted ϵ = 0.02455354787220282, std = [1.0, 1.0, 1.0, 1.0, 1.0]; 1000 iterations is used for adaption.
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/adapt/adapt.jl:91
[NUTS] Finished with
  Running time        = 250.91819286099994;
  #lf / sample        = 0.0;
  #evals / sample     = 47.296;
  pre-cond. metric    = [1.0, 1.0, 1.0, 1.0, 1.0].


Describe the posterior samples

In [10]:
describe(posterior)

Iterations = 1:2000
Thinning interval = 1
Chains = 1
Samples per chain = 2000

Empirical Posterior Estimates:
              Mean              SD            Naive SE          MCSE          ESS    
       α    9.181455714  3.933350958×10⁻¹    0.0087952401    0.0367816441  114.357097
  lf_num    0.000000000                 0    0.0000000000    0.0000000000         NaN
      βA   -1.888066517  4.566855972×10⁻¹    0.0102118004    0.0440006905  107.724787
      βR   -0.170404369  3.335329574×10⁻¹    0.0074580237    0.0316124718  111.316771
       σ 1324.048654985    5.43520801×10⁴ 1215.3494579162 1323.0977011788 1687.518370
 elapsed    0.125459096 7.9970404761×10⁻²    0.0017881926    0.0021390520 1397.706251
 epsilon    0.027552754 1.7361337791×10⁻²    0.0003882113    0.0007902756  482.624344
eval_num   47.296000000    2.78267416×10¹    0.6222248580    0.7205708264 1491.321779
     βAR    0.390309023  1.700210609×10⁻¹    0.0038017865    0.0060423862  791.750743
      lp -258.808577498    1.2

Fix the inclusion of adaptation samples

In [11]:
posterior2 = MCMCChain.Chains(posterior.value[1001:2000,:,:], names=posterior.names)

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1
Samples per chain = 1000

Union{Missing, Float64}[9.19034 0.0 … -249.045 0.0245535; 9.16767 0.0 … -247.055 0.0245535; … ; 9.29008 0.0 … -248.106 0.0245535; 9.26406 0.0 … -246.688 0.0245535]

Example of a Turing run simulation output

Here's the ulam() output from rethinking

In [12]:
m8_1s_cmdstan = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD       Naive SE       MCSE      ESS
    a  9.22360053 0.139119116 0.0021996664 0.0034632816 1000
   bR -0.20196346 0.076106388 0.0012033477 0.0018370185 1000
   bA -1.94430980 0.227080488 0.0035904578 0.0057840746 1000
  bAR  0.39071684 0.131889143 0.0020853505 0.0032749642 1000
sigma  0.95036370 0.052161768 0.0008247500 0.0009204073 1000

Quantiles:
          2.5%       25.0%       50.0%      75.0%        97.5%
    a  8.95307475  9.12719750  9.2237750  9.31974000  9.490234250
   bR -0.35217930 -0.25334425 -0.2012855 -0.15124725 -0.054216855
   bA -2.39010825 -2.09894500 -1.9432550 -1.78643000 -1.513974250
  bAR  0.13496995  0.30095575  0.3916590  0.47887625  0.650244475
sigma  0.85376115  0.91363250  0.9484920  0.98405750  1.058573750
";

Describe the posterior samples

In [13]:
describe(posterior2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1
Samples per chain = 1000

Empirical Posterior Estimates:
              Mean                   SD                       Naive SE              MCSE         ESS   
       α    9.220223898  0.136246523139085862208475 0.0043084933639834536653335 0.0091337309  222.51251
  lf_num    0.000000000  0.000000000000000000000000 0.0000000000000000000000000 0.0000000000        NaN
      βA   -1.939239358  0.226602042770951583872119 0.0071657857760309981789981 0.0193795497  136.72259
      βR   -0.204729891  0.078700402464217750209130 0.0024887252455885627323851 0.0038821380  410.97174
       σ    0.952001663  0.054100018068588974495814 0.0017107927855300457831850 0.0012247892 1000.00000
 elapsed    0.123533711  0.071055947788420378841145 0.0022469863631341254958662 0.0018691656 1000.00000
 epsilon    0.024553548  0.000000000000000041654196 0.0000000000000000013172213 0.0000000000 1000.00000
eval_num   48.052000000 27.096780833367102303554930 0.8568

end of 08/m8.1t.jl#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*